In [4]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
from datetime import datetime

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [5]:
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [6]:
from torchvision import datasets, transforms
data_path = 'C:/Users/nrashvan/data'

cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [7]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out
    
model = Net()

learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

n_epochs = 300

for epoch in range(n_epochs+1):
    for imgs, labels in train_loader:
        outputs = model(imgs)
        loss = loss_fn(outputs, labels)
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch == 0 or epoch % 10 == 0:
        now = datetime.now()
        dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
        print(dt_string, "Epoch: %d, Loss: %f" % (epoch, float(loss)))

09/12/2022 16:20:30 Epoch: 0, Loss: 1.726280
09/12/2022 16:25:15 Epoch: 10, Loss: 1.236104
09/12/2022 16:29:58 Epoch: 20, Loss: 0.955580
09/12/2022 16:34:29 Epoch: 30, Loss: 0.896548
09/12/2022 16:38:56 Epoch: 40, Loss: 0.369287
09/12/2022 16:43:24 Epoch: 50, Loss: 0.630693
09/12/2022 16:47:49 Epoch: 60, Loss: 1.261155
09/12/2022 16:52:16 Epoch: 70, Loss: 1.314162
09/12/2022 16:56:47 Epoch: 80, Loss: 0.496648
09/12/2022 17:01:17 Epoch: 90, Loss: 0.508200
09/12/2022 17:05:43 Epoch: 100, Loss: 0.808376
09/12/2022 17:10:10 Epoch: 110, Loss: 1.312326
09/12/2022 17:14:35 Epoch: 120, Loss: 0.540730
09/12/2022 17:18:58 Epoch: 130, Loss: 0.689347
09/12/2022 17:23:25 Epoch: 140, Loss: 1.002603
09/12/2022 17:27:51 Epoch: 150, Loss: 0.608997
09/12/2022 17:32:17 Epoch: 160, Loss: 0.717593
09/12/2022 17:36:40 Epoch: 170, Loss: 0.323345
09/12/2022 17:41:07 Epoch: 180, Loss: 0.562225
09/12/2022 17:45:33 Epoch: 190, Loss: 0.653944
09/12/2022 17:49:58 Epoch: 200, Loss: 0.785235
09/12/2022 17:54:23 Epoc

In [10]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        outputs = model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))


Accuracy: 0.814600
